# Text search

> Quick get started with the gallery module

### TextSearch use case

In [2]:
from vespa.gallery import TextSearch

app_package = TextSearch(id_field="id", text_fields=["title", "body"])

### Deploy

In [3]:
from vespa.deployment import VespaDocker

disk_folder = os.path.join(os.getenv("WORK_DIR"), "sample_application")
vespa_docker = VespaDocker(port=8089, disk_folder=disk_folder)
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for application status.
Finished deployment.


### Feed

In [4]:
from pandas import read_csv

docs = read_csv(
    filepath_or_buffer="https://data.vespa.oath.cloud/blog/msmarco/sample_docs.csv"
)
docs.head()

,id,title,body
0,D1712962,Can you eat crab or imitation krab when you ha...,Answers com Wiki Answers Categories Health...
1,D1817294,How long is a tax refund check good,Answers com Wiki Answers Categories Busine...
2,D1761039,The Suffolk Resolves 1774,The Suffolk Resolves 1774 Across New England ...
3,D2899268,The eagle has flown,Download citation Share Download full text PDF...
4,D3278481,22b Cotton and African American Life,22b Cotton and African American Life Two thi...


In [5]:
responses = app.feed_df(docs)

### Query

In [7]:
response = app.query(query="what keeps planes in the air")

In [8]:
for hit in response.hits:
    print({
        "id": hit["fields"]["id"], 
        "title": hit["fields"]["title"], 
        "relevance": hit["relevance"]
    })

{'id': 'D1871659', 'title': 'What keeps airplanes in the air ', 'relevance': 25.62689087145899}
{'id': 'D684487', 'title': 'MP02  Motion Diagrams', 'relevance': 10.529337096665039}
{'id': 'D254873', 'title': ' ', 'relevance': 9.359158504016035}
{'id': 'D1765332', 'title': 'Fast   Furious 6', 'relevance': 8.073806736512351}
{'id': 'D1631044', 'title': 'First Cell', 'relevance': 7.633196961117181}
{'id': 'D3434656', 'title': 'A Global Guide to Pet Relocation Costs', 'relevance': 6.923486758052908}
{'id': 'D3157544', 'title': 'Thomas Cook to axe 2 600 jobs', 'relevance': 6.619466866937314}
{'id': 'D209851', 'title': 'The USS Scorpion Buried at Sea', 'relevance': 6.405321666303772}
{'id': 'D958861', 'title': 'George Harrison  close to death ', 'relevance': 6.007963343935413}
{'id': 'D1529248', 'title': 'The airport shopping challenge  Left buying Christmas presents til  the last minute like I did  You CAN do it all in two hours   and I m living proof', 'relevance': 5.990555132518157}
